In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys
from PIL import Image

## Load data

In [ ]:
from libtiff import TIFF
# to open a tiff file for reading:
tif = TIFF.open('../input/seg/test/' + '35811095.tif', mode='r')

# to read an image in the currect TIFF directory and return it as numpy array:
image = tif.read_image()

In [ ]:
def get_patch(img_arr, size, stride):
    
    patches_list = []
    overlapping = 0

    ind_1 = image.shape[0] // size
    ind_2 = image.shape[1] // size
    k = 0
    l = 0
    for i in range(ind_1+1):
        
        for j in range(ind_2+1):
            print(i, ind_1, j, ind_2)
            if (i == ind_1) and (j != ind_2):
                
                pat = 255 * np.ones((size-(image.shape[0] - ind_1*size), size, 3))
                pat = np.asarray(pat, dtype=np.uint8)
                img_ = np.concatenate((image[ind_1 * size : image.shape[0], k * size : k * size +stride], pat), axis=0)
                patches_list.append(img_)
                k +=1

            elif (j == ind_2) and (i != ind_1):
        
                pat2 = 255 * np.ones((size, size-(image.shape[1] - ind_2*size), 3))
                pat2 = np.asarray(pat2, dtype=np.uint8)
                img_2 = np.concatenate((image[l * size : l * size +stride, ind_2 * size : image.shape[1]], pat2), axis =1)
                patches_list.append(img_2)
                l +=1

            elif (i == ind_1) and (j == ind_2):
                
                pat3 = 255 * np.ones((image.shape[0] - ind_1*size, size-(image.shape[1] - ind_2*size), 3))
                pat3 = np.asarray(pat3, dtype=np.uint8)
                img_3 = np.concatenate((image[ind_1 * size : image.shape[0], ind_2 * size : image.shape[1]], pat3), axis =1)
                pat4 = 255 * np.ones((size-(image.shape[0] - ind_1*size), size, 3))
                pat4 = np.asarray(pat4, dtype=np.uint8)
                img_4 = np.concatenate((img_3, pat4), axis =0)
                patches_list.append(img_4)
                
            else:
                patches_list.append(
                    img_arr[
                        i * stride : i * stride + size,
                        j * stride : j * stride + size
                    ]
                )
            
                        
    return np.stack(patches_list)

In [ ]:
x_crops = get_patch(img_arr=image, size=1024, stride=1024) 

In [ ]:
x_test = np.asarray(x_crops, dtype=np.float32)/255
print(x_test.shape)

In [ ]:
x_test[0]

In [ ]:
# masks = glob.glob("../input/seg/*.png")
masks_test = glob.glob("../input/seg/*.png")
# orgs = list(map(lambda x: x.replace(".png", ".jpg"), masks))
orgs_test = list(map(lambda x: x.replace(".png", ".jpg"), masks_test))

In [ ]:
# imgs_test_list = []
# imgs_test_list.append(np.array(Image.open("../input/seg/test01/277.jpg").resize((384,384))))

In [ ]:
# imgs_test_np = np.asarray(imgs_test_list)

In [ ]:
# x_test = np.asarray(imgs_test_np, dtype=np.float32)/255

In [ ]:
# print(imgs_test_np.shape)

In [ ]:
# imgs_list = []
# masks_list = []
imgs_test_list = []
masks_test_list = []
# for image, mask in zip(orgs, masks):
#     imgs_list.append(np.array(Image.open(image).resize((384,384))))
#     masks_list.append(np.array(Image.open(mask).resize((384,384))))
    
for image, mask in zip(orgs_test, masks_test):
    imgs_test_list.append(np.array(Image.open(image).resize((384,384))))
    masks_test_list.append(np.array(Image.open(mask).resize((384,384))))

# imgs_np = np.asarray(imgs_list)
# masks_np = np.asarray(masks_list)
imgs_test_np = np.asarray(imgs_test_list)
masks_test_np = np.asarray(masks_test_list)

In [ ]:
# print(imgs_np.shape, masks_np.shape, imgs_test_np.shape, masks_test_np.shape)
print(imgs_test_np.shape, masks_test_np.shape)

## Plot images + masks + overlay (mask over original)

In [ ]:
from keras_unet.utils import plot_imgs

# plot_imgs(org_imgs=imgs_np, mask_imgs=masks_np, nm_img_to_plot=10, figsize=6)
plot_imgs(org_imgs=imgs_test_np, mask_imgs=masks_test_np, nm_img_to_plot=10, figsize=6)

## Get data into correct shape, dtype and range (0.0-1.0)

In [ ]:
# print(imgs_np.max(), masks_np.max())
print(imgs_test_np.max(), masks_test_np.max())

In [ ]:
# x = np.asarray(imgs_np, dtype=np.float32)/255
# y = np.asarray(masks_np, dtype=np.float32)
x_test = np.asarray(imgs_test_np, dtype=np.float32)/255
y_test = np.asarray(masks_test_np, dtype=np.float32)

In [ ]:
# print(x.max(), y.max())
print(x_test.max(), y_test.max())

In [ ]:
# print(x.shape, y.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# y = y.reshape(y.shape[0], y.shape[1], y.shape[2], 1)
# print(x.shape, y.shape)
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1], y_test.shape[2], 1)
print(x_test.shape, y_test.shape)

##  Train/val split

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=0)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

## Prepare train generator with data augmentation

In [ ]:
from keras_unet.utils import get_augmented

train_gen = get_augmented(
    x_train, y_train, batch_size=2,
    data_gen_args = dict(
        rotation_range=5.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=40,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='constant'
    ))

In [ ]:
sample_batch = next(train_gen)
xx, yy = sample_batch
print(xx.shape, yy.shape)
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=xx, mask_imgs=yy, nm_img_to_plot=2, figsize=6)

## Initialize network

In [ ]:
from keras_unet.models import custom_unet

input_shape = x_train[0].shape

model = custom_unet(
    input_shape,
    filters=32,
    use_batch_norm=True,
    dropout=0.3,
    dropout_change_per_layer=0.0,
    num_layers=4
)

In [ ]:
model.summary()

## Compile + train

In [ ]:
from keras.callbacks import ModelCheckpoint


model_filename = 'seg_model_v3.h5'
callback_checkpoint = ModelCheckpoint(
    model_filename, 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True,
)

In [ ]:
from keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

model.compile(
    optimizer=Adam(), 
    #optimizer=SGD(lr=0.01, momentum=0.99),
    loss='binary_crossentropy',
    #loss=jaccard_distance,
    metrics=[iou, iou_thresholded]
)


In [ ]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=200,
    epochs=50,
    
    validation_data=(x_val, y_val),
    callbacks=[callback_checkpoint]
)

## Plot training history

In [ ]:
from keras_unet.utils import plot_segm_history

plot_segm_history(history)

## Plot original + ground truth + pred + overlay (pred on top of original)

In [ ]:
from keras.models import load_model
from keras_unet.metrics import iou, iou_thresholded
weights_path = os.path.join("../notebooks", "seg_model_v3_1024.h5")
model = load_model(weights_path, custom_objects={"iou":iou,"iou_thresholded":iou_thresholded})
# model.load_weights(model_filename)
y_test_pred = model.predict(x_test[0:2])

In [ ]:
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=x_test, mask_imgs=y_test, pred_imgs=y_test_pred, nm_img_to_plot=10)

In [ ]:
plt.imshow(y_test_pred[0,:,:,0])

In [ ]:
import cv2

for i in range(y_test_pred.shape[0]):
    for j in range(y_test_pred[i].shape[0]):
        for k in range(y_test_pred[i].shape[1]):
            if y_test_pred[i,j,k] > 0.1:
                y_test_pred[i,j,k] = 255
           
            else:
                y_test_pred[i,j,k] = 0
      
    plt.imshow(y_test_pred[i,:,:,0])

In [ ]:
y_test_pred = y_test_pred.astype(np.uint8)
y_test_pred.shape

In [ ]:
import cv2

for j in range(y_test_pred.shape[0]):
    
    img =y_test_pred[j].astype(np.uint8)
    # imgray = cv2.cvtColor(img,cv2.COLOR_GRAY2GRAY)
    ret, thresh = cv2.threshold(img[:,:,0],127,255,0)
    image, contours, hierachy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    area_list = []
    mean_value = []
    k=0
    for i in range(len(contours)):
        if len(contours[i]) > 1 and cv2.contourArea(contours[i]) > 100:
            area_list.append(cv2.contourArea(contours[i]))
            x0, y0 = zip(*np.squeeze(contours[i]))
            x0_mean =np.mean(x0)
            y0_mean =np.mean(y0)
            plt.plot(x0, y0, c="b")
            k+=1
            plt.plot(x0_mean, y0_mean, marker='$%d$'%k, markersize=10)
            print('ID = %d'%k, 'Area = %.2f'%cv2.contourArea(contours[i]), int(cv2.contourArea(contours[i])))
            
#             h = y_test_pred[j].shape[0]
#             w = y_test_pred[j].shape[1]
#             zeros = np.zeros((h, w))
#             zeros[x0, y0] = 255

#             ones = zeros
#             zeros = np.zeros((h, w))
#             new_ = np.stack((ones, zeros, zeros, ones), axis=-1)
#             plt.imshow(new_)


    h = y_test_pred[j].shape[0]
    w = y_test_pred[j].shape[1]
    zeros = np.zeros((h, w))
    ones = y_test_pred[j].reshape(h, w)
    new = np.stack((ones, zeros, zeros, ones), axis=-1)

    plt.imshow(imgs_test_np[j,:,:])
    plt.imshow(new, alpha=0.3)

#     plt.imshow(y_test_pred[j,:,:,0])
    plt.xlim(0, 384)
    plt.ylim(384, 0)
    plt.show()
#     plt.savefig('test_%d.png'%j, dpi = 300)
    plt.clf()

In [ ]:
len(cont)

In [ ]:
plt.plot(cont[1][0],cont[1][1], c="b")

In [ ]:
h = y_test_pred[j].shape[0]
w = y_test_pred[j].shape[1]
zeros = np.zeros((h, w))
zeros[x0, y0] = 255

ones = zeros
zeros = np.zeros((h, w))
new_ = np.stack((ones, zeros, zeros, ones), axis=-1)
plt.imshow(new_)

In [ ]:
# plt.imshow(imgs_test_np[0,:,:])
for i in range(len(contours)):
    if len(contours[i]) > 1 and cv2.contourArea(contours[i]) > 100:
        area_list.append(cv2.contourArea(contours[i]))
        x0, y0 = zip(*np.squeeze(contours[i]))
#             plt.plot(x0, y0, c="b")

        h = y_test_pred[j].shape[0]
        w = y_test_pred[j].shape[1]
        zeros = np.zeros((h, w))
        zeros[x0, y0] = 255

        ones = zeros
        zeros = np.zeros((h, w))
        new_ = np.stack((ones, zeros, zeros, ones), axis=0)
        plt.imshow(new_)


In [ ]:
plt.imshow(imgs_test_np[0,:,:])

In [ ]:
plt.imshow(new_)

In [ ]:
from skimage import io
width = 500
height = 500
bpp = 3

img = np.zeros((height, width, bpp), np.uint8)


# test1 = cv.rectangle(img, (50, 50),  (450, 450), (0, 0, 255), 3)

test2 = cv2.rectangle(img, (150, 200), (250, 300), (255, 255, 255), -1)
test2 = cv2.line(img, (20, 80), (120, 180), (255, 255, 255))
test2 = cv2.line(img, (20, 80), (20, 180), (255, 255, 255))
test2 = cv2.line(img, (20, 180), (120, 180), (255, 255, 255))
print(test2.shape)
io.imsave('test.png',test2)
io.imshow(test2)

In [ ]:
img = io.imread('test.png')
print(img.shape)
ret, thresh = cv2.threshold(img[:,:,0],127,255,0)
image, contours, hierachy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [ ]:
print(cv2.contourArea(contours[0]))
x0, y0 = zip(*np.squeeze(contours[0]))
plt.plot(x0, y0, c="b")

In [ ]:
print(cv2.contourArea(contours[1]))
x0, y0 = zip(*np.squeeze(contours[1]))
plt.plot(x0, y0, c="b")